# Fixed Database Insertion Script - All Errors Resolved

This notebook properly handles:
- Phone number truncation
- Foreign key relationships (by mapping IDs back from the DB)
- Column mapping issues
- Identity column handling and table cleanup for repeatable loads

## Setup and Database Connection

In [68]:
import pandas as pd
from sqlalchemy import create_engine, text
import numpy as np
import warnings
import traceback
warnings.filterwarnings('ignore')

# Database connection
engine = create_engine(
    "mssql+pyodbc://localhost\\SQLEXPRESS/Book_haven?"
    "driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

In [ ]:
# Test connection
try:
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("Database connected successfully!")
except Exception as e:
    print(f"Connection failed: {e}")

✅ Database connected successfully!


## PRE-RUN CLEANUP: Clear All Tables and Reset Identity

This is essential to avoid Primary Key and Unique Constraint violations on re-run.

In [ ]:
try:
    with engine.begin() as conn:
        # Delete children first due to Foreign Key constraints
        conn.execute(text("DELETE FROM reservation_details"))
        conn.execute(text("DELETE FROM reservation"))
        conn.execute(text("DELETE FROM book_author"))
        conn.execute(text("DELETE FROM book_category"))
        conn.execute(text("DELETE FROM book_copy"))
        conn.execute(text("DELETE FROM book"))
        conn.execute(text("DELETE FROM staff"))
        conn.execute(text("DELETE FROM member"))
        conn.execute(text("DELETE FROM category"))
        conn.execute(text("DELETE FROM author"))
        conn.execute(text("DELETE FROM description"))
        
        # Reset Identity/Auto-increment counters for all tables
        conn.execute(text("DBCC CHECKIDENT ('description', RESEED, 0)"))
        conn.execute(text("DBCC CHECKIDENT ('author', RESEED, 0)"))
        conn.execute(text("DBCC CHECKIDENT ('category', RESEED, 0)"))
        conn.execute(text("DBCC CHECKIDENT ('member', RESEED, 0)"))
        conn.execute(text("DBCC CHECKIDENT ('staff', RESEED, 0)"))
        conn.execute(text("DBCC CHECKIDENT ('book_copy', RESEED, 0)"))
        conn.execute(text("DBCC CHECKIDENT ('reservation', RESEED, 0)"))
        # reservation_details usually doesn't have an identity column
        
        print("Database tables cleared and Identity counters reset.")
except Exception as e:
    print(f"Cleanup failed: {e}")

✅ Database tables cleared and Identity counters reset.


## Step 1: Insert Independent Tables (No Foreign Keys)

### 1.1 Description Table

In [ ]:
try:
    description = pd.read_csv("data/description.csv")
    description = description[['description']].copy()
    description.dropna(subset=['description'], inplace=True)
    description.to_sql('description', con=engine, if_exists='append', index=False)
    print(f"Inserted {len(description)} descriptions")
except Exception as e:
    print(f"Description insertion failed: {e}")

✅ Inserted 2500 descriptions


### 1.2 Author Table

In [ ]:
try:
    author = pd.read_csv("data/author.csv")
    author = author[['name']].copy()
    author['name'] = author['name'].str.strip()
    author.dropna(subset=['name'], inplace=True)
    author.drop_duplicates(subset=['name'], inplace=True)
    author.to_sql('author', con=engine, if_exists='append', index=False)
    print(f" Inserted {len(author)} authors")
except Exception as e:
    print(f" Author insertion failed: {e}")

✅ Inserted 4460 authors


### 1.3 Category Table

In [73]:
try:
    category = pd.read_csv("data/category.csv")
    category = category[['category_name']].copy()
    category['category_name'] = category['category_name'].str.strip()
    category.dropna(subset=['category_name'], inplace=True)
    category.drop_duplicates(subset=['category_name'], inplace=True)
    category.to_sql('category', con=engine, if_exists='append', index=False)
    print(f"✅ Inserted {len(category)} categories")
except Exception as e:
    print(f"❌ Category insertion failed: {e}")

✅ Inserted 802 categories


### 1.4 Member Table - WITH PHONE TRUNCATION FIX

In [ ]:
try:
    member = pd.read_csv("data/member.csv")
    member_cols = ['fname', 'lname', 'phone', 'email', 'city', 'street', 'bdate']
    member = member[member_cols].copy()
    
    # FIX: Truncate phone numbers to 20 characters (matching SQL schema)
    member['phone'] = member['phone'].astype(str).str[:20]
    
    member.dropna(subset=['fname', 'lname', 'email'], inplace=True)
    member.drop_duplicates(subset=['email'], inplace=True)
    member.to_sql('member', con=engine, if_exists='append', index=False)
    print(f"Inserted {len(member)} members")
except Exception as e:
    print(f"Member insertion failed: {e}")

✅ Inserted 3000 members


### 1.5 Staff Table - WITH PHONE TRUNCATION FIX

In [ ]:
try:
    staff = pd.read_csv("data/staff.csv")
    staff_cols = ['fname', 'lname', 'phone', 'email', 'role', 'password', 'salary']
    staff = staff[staff_cols].copy()
    
    # FIX: Truncate phone numbers to 20 characters (matching SQL schema)
    staff['phone'] = staff['phone'].astype(str).str[:20]
    
    staff.dropna(subset=['fname', 'lname', 'email', 'password'], inplace=True)
    staff.drop_duplicates(subset=['email'], inplace=True)
    staff.to_sql('staff', con=engine, if_exists='append', index=False)
    print(f"Inserted {len(staff)} staff members")
except Exception as e:
    print(f"Staff insertion failed: {e}")

✅ Inserted 100 staff members


## Step 2: Retrieve Auto-Generated IDs (The FK Mapping Step)

This is the critical step! We need to query the actual Primary Keys generated by SQL Server to correctly map Foreign Keys in the next steps.

In [ ]:
try:
    # Get description IDs
    with engine.connect() as conn:
        desc_map = pd.read_sql("SELECT description_id, description FROM description", conn)
    print(f"Retrieved {len(desc_map)} description IDs")
    
    # Get author IDs
    with engine.connect() as conn:
        author_map = pd.read_sql("SELECT author_id, name FROM author", conn)
    print(f"Retrieved {len(author_map)} author IDs")
    
    # Get category IDs
    with engine.connect() as conn:
        category_map = pd.read_sql("SELECT category_id, category_name FROM category", conn)
    print(f"Retrieved {len(category_map)} category IDs")
    
    # Get member IDs
    with engine.connect() as conn:
        member_map = pd.read_sql("SELECT member_id, email FROM member", conn)
    print(f"Retrieved {len(member_map)} member IDs")
    
    # Get staff IDs
    with engine.connect() as conn:
        staff_map = pd.read_sql("SELECT staff_id, email FROM staff", conn)
    print(f"Retrieved {len(staff_map)} staff IDs")
    
except Exception as e:
    print(f"Failed to retrieve IDs: {e}")
    traceback.print_exc()

✅ Retrieved 2500 description IDs
✅ Retrieved 4460 author IDs
✅ Retrieved 802 category IDs
✅ Retrieved 3000 member IDs
✅ Retrieved 100 staff IDs


## Step 3: Insert Book Table (FK to description)

In [ ]:
try:
    book = pd.read_csv("data/book.csv")
    book_clean = book[['ISBN', 'title', 'publication_year']].copy()
    
    # ASSUMPTION: description IDs match the insertion order of the book rows in the CSV.
    if len(book_clean) <= len(desc_map):
        book_clean['description_id'] = desc_map['description_id'].iloc[:len(book_clean)].values
    else:
        print("Warning: More books than descriptions! Using only available IDs.")
        book_clean = book_clean.iloc[:len(desc_map)]
        book_clean['description_id'] = desc_map['description_id'].values
    
    # Clean and convert year column
    book_clean['publication_year'] = pd.to_numeric(
        book_clean['publication_year'], errors='coerce'
    ).astype('Int64')
    
    book_clean.dropna(subset=['ISBN', 'title'], inplace=True)
    book_clean.drop_duplicates(subset=['ISBN'], inplace=True)
    
    book_clean.to_sql('book', con=engine, if_exists='append', index=False)
    print(f"Inserted {len(book_clean)} books")
    
    # Retrieve existing ISBNs for subsequent FK checks
    with engine.connect() as conn:
        existing_isbns = pd.read_sql("SELECT ISBN FROM book", conn)
    
except Exception as e:
    print(f"Book insertion failed: {e}")
    traceback.print_exc()

✅ Inserted 2500 books


## Step 4: Insert Junction Tables

### 4.1 Book_Author Table - FIXED COLUMN MAPPING (FK to book and author)

In [ ]:
import sqlalchemy

try:
    book_author = pd.read_csv("data/book_author.csv")
    
    # 1. Determine the source column name
    if 'Authors_list' in book_author.columns:
        author_col = 'Authors_list'
    elif 'author_id' in book_author.columns: 
        author_csv = pd.read_csv("data/author.csv")
        old_author_map = author_csv[['author_id', 'name']].drop_duplicates()
        book_author = book_author.merge(
            old_author_map,
            on='author_id',
            how='inner'
        )
        author_col = 'name'
    else:
        raise ValueError(f"Cannot find author mapping column in book_author CSV")
    
    # Pre-merge cleaning
    book_author[author_col] = book_author[author_col].astype(str).str.strip()
    
    # Ensure the author_map is also perfectly clean
    author_map['name'] = author_map['name'].astype(str).str.strip()
    
    # Perform the INNER join to map to the database author_id
    # No need for diagnostic code here as the problem is identified
    book_author_clean = book_author.merge(
        author_map.rename(columns={'author_id': 'db_author_id'}), # Rename to avoid conflicts
        left_on=author_col,
        right_on='name',
        how='inner'
    )
    
    # Select final columns and rename
    book_author_clean = book_author_clean[['ISBN', 'db_author_id']].copy()
    book_author_clean.rename(columns={'db_author_id': 'author_id'}, inplace=True)
    
    # Final data cleaning steps (deduplication and NaN removal)
    book_author_clean.dropna(subset=['ISBN', 'author_id'], inplace=True)
    book_author_clean.drop_duplicates(subset=['ISBN', 'author_id'], inplace=True)
    
    # Verify ISBNs exist in book table (FK check)
    book_author_clean = book_author_clean[
        book_author_clean['ISBN'].isin(existing_isbns['ISBN'])
    ]
    
    book_author_clean['author_id'] = book_author_clean['author_id'].astype('Int64')
    
    # --- FIX FOR DuplicateColumnError ---
    # Define the data types explicitly to prevent SQLAlchemy from creating conflicting schema
    dtype_mapping = {
        'ISBN': sqlalchemy.types.VARCHAR(13),  # Adjust size if your schema is different
        'author_id': sqlalchemy.types.Integer 
    }
    
    # Final insertion
    if len(book_author_clean) > 0:
        book_author_clean.to_sql(
            'book_author', 
            con=engine, 
            if_exists='append', 
            index=False,
            dtype=dtype_mapping # <-- The fix
        )
        print(f"Inserted {len(book_author_clean)} book-author relationships")
    else:
        print("❌ Final DataFrame is empty. No rows inserted.")
    
except Exception as e:
    print(f"book_author insertion failed: {e}")
    traceback.print_exc()

✅ Inserted 6450 book-author relationships


### 4.2 Book_Category Table - FIXED COLUMN MAPPING (FK to book and category)

In [ ]:
try:
    book_category = pd.read_csv("data/book_category.csv")
    
    # Read the original category data to get the mapping from old ID to Name
    category_csv = pd.read_csv("data/category.csv")
    
    # Ensure the database map is clean
    category_map['category_name'] = category_map['category_name'].astype(str).str.strip()
    
    # --- DIAGNOSTIC PASS (Retained) ---
    # (The successful diagnostic code has been condensed for final insertion)
    
    if 'category_id' in book_category.columns:
        old_cat_map = category_csv[['category_id', 'category_name']].drop_duplicates()
        
        # 1. Map old ID to Name
        book_category_with_names = book_category.merge(
            old_cat_map,
            on='category_id',
            how='inner'
        )
        book_category_with_names['category_name'] = book_category_with_names['category_name'].astype(str).str.strip()
        
        # 2. Map Name to New DB ID
        diagnostic_merge = book_category_with_names.merge(
            category_map.rename(columns={'category_id': 'db_category_id'}),
            on='category_name',
            how='inner'
        )
        
        # Final DataFrame preparation
        book_category_clean = diagnostic_merge[['ISBN', 'db_category_id']].copy()
        book_category_clean.rename(columns={'db_category_id': 'category_id'}, inplace=True)
        book_category_clean['category_id'] = book_category_clean['category_id'].astype('Int64')
        
    else:
        raise ValueError("Cannot find category_id column in book_category CSV")
    
    book_category_clean.dropna(subset=['ISBN', 'category_id'], inplace=True)
    book_category_clean.drop_duplicates(subset=['ISBN', 'category_id'], inplace=True)
    
    # Verify ISBNs exist
    book_category_clean = book_category_clean[
        book_category_clean['ISBN'].isin(existing_isbns['ISBN'])
    ]
    
    # --- FIX FOR DuplicateColumnError ---
    # Define the data types explicitly to prevent SQLAlchemy from creating conflicting schema
    dtype_mapping = {
        'ISBN': sqlalchemy.types.VARCHAR(13),  # Assuming ISBN is VARCHAR(13)
        'category_id': sqlalchemy.types.Integer 
    }
    
    # Final insertion
    if len(book_category_clean) > 0:
        book_category_clean.to_sql(
            'book_category', 
            con=engine, 
            if_exists='append', 
            index=False,
            dtype=dtype_mapping # <-- The fix
        )
        print(f"Inserted {len(book_category_clean)} book-category relationships")
    else:
        print("Final DataFrame is empty. No rows inserted.")
    
except Exception as e:
    print(f"book_category insertion failed: {e}")
    traceback.print_exc()

✅ Inserted 5964 book-category relationships


## Step 5: Insert Book_Copy (FK to book)

In [ ]:
try:
    # FIX: Get existing ISBNs fresh (in case previous cell didn't run or update the variable)
    with engine.connect() as conn:
        existing_isbns = pd.read_sql("SELECT ISBN FROM book", conn)
    
    book_copy = pd.read_csv("data/book_copy.csv")
    book_copy_cols = ['status', 'condition', 'price', 'ISBN']
    book_copy_clean = book_copy[book_copy_cols].copy()
    
    # Verify ISBNs exist (FK check)
    book_copy_clean = book_copy_clean[
        book_copy_clean['ISBN'].isin(existing_isbns['ISBN'])
    ]
    
    book_copy_clean.dropna(subset=['ISBN'], inplace=True)
    book_copy_clean.to_sql('book_copy', con=engine, if_exists='append', index=False)
    print(f"Inserted {len(book_copy_clean)} book copies")
    
except Exception as e:
    print(f"book_copy insertion failed: {e}")
    traceback.print_exc()

✅ Inserted 5000 book copies


## Step 6: Insert Reservation and Reservation Details

### 6.1 Reservation Table - FIXED MAPPING (FK to member and staff)

In [81]:
try:
    if len(member_map) == 0 or len(staff_map) == 0:
        print("⚠️ Cannot insert reservations: No members or staff in database")
    else:
        reservation = pd.read_csv("data/reservation.csv")
        member_csv = pd.read_csv("data/member.csv")
        staff_csv = pd.read_csv("data/staff.csv")
        
        # Prepare mapping from old CSV IDs to Email (the shared unique identifier)
        member_csv['phone'] = member_csv['phone'].astype(str).str[:20]
        old_member_map = member_csv[['member_id', 'email']].drop_duplicates()
        
        staff_csv['phone'] = staff_csv['phone'].astype(str).str[:20]
        old_staff_map = staff_csv[['staff_id', 'email']].drop_duplicates()
        
        # --- Member Mapping ---
        reservation_with_email = reservation.merge(
            old_member_map,
            on='member_id',
            how='inner'
        )
        
        # Merge with actual database member IDs (member_map)
        reservation_clean = reservation_with_email.merge(
            member_map,
            on='email',
            how='inner',
            suffixes=('_old', '_new')
        )
        
        # --- Staff Mapping ---
        # Use email as the bridge for staff mapping as well
        reservation_clean = reservation_clean.merge(
            old_staff_map.rename(columns={'staff_id': 'staff_id_old', 'email': 'staff_email'}),
            left_on='staff_id',
            right_on='staff_id_old',
            how='inner'
        )
        
        reservation_clean = reservation_clean.merge(
            staff_map.rename(columns={'email': 'staff_email', 'staff_id': 'staff_id_new'}),
            on='staff_email',
            how='inner'
        )
        
        # Select final columns
        res_final = reservation_clean[[
            'member_id_new', 'staff_id_new', 'reservation_date', 
            'expiration_date', 'returned_at'
        ]].copy()
        
        res_final.rename(columns={
            'member_id_new': 'member_id',
            'staff_id_new': 'staff_id'
        }, inplace=True)
        
        res_final.to_sql('reservation', con=engine, if_exists='append', index=False)
        print(f"✅ Inserted {len(res_final)} reservations")
        
except Exception as e:
    print(f"❌ Reservation insertion failed: {e}")
    traceback.print_exc()

✅ Inserted 4503 reservations


### 6.2 Reservation Details Table (FK to reservation and book_copy)

In [82]:
try:
    # Get new reservation IDs from database
    with engine.connect() as conn:
        db_reservations = pd.read_sql("SELECT reservation_id, member_id, reservation_date FROM reservation", conn)
    print(f"Retrieved {len(db_reservations)} reservation IDs from database")
    
    if len(db_reservations) == 0:
        print("⚠️ No reservations in database, skipping reservation_details")
    else:
        # Get book_copy IDs
        with engine.connect() as conn:
            copy_map = pd.read_sql("SELECT copy_id FROM book_copy", conn)
        
        res_details = pd.read_csv("data/reservation_details.csv")
        
        # ASSUMPTION: Map old reservation_id to new reservation_id by sequence/order
        old_res_ids = sorted(res_details['reservation_id'].unique())
        new_res_ids = db_reservations['reservation_id'].tolist()
        
        if len(old_res_ids) <= len(new_res_ids):
            res_id_mapping = dict(zip(old_res_ids, new_res_ids[:len(old_res_ids)]))
            
            res_details_clean = res_details.copy()
            res_details_clean['reservation_id'] = res_details_clean['reservation_id'].map(res_id_mapping)
            
            # Filter valid copy_ids and reservation_ids (FK checks)
            res_details_clean = res_details_clean[
                res_details_clean['copy_id'].isin(copy_map['copy_id']) &
                res_details_clean['reservation_id'].notna()
            ]
            
            res_details_clean.drop_duplicates(subset=['reservation_id', 'copy_id'], inplace=True)
            
            res_details_clean.to_sql(
                'reservation_details', 
                con=engine, 
                if_exists='append', 
                index=False
            )
            print(f"✅ Inserted {len(res_details_clean)} reservation details")
        else:
            print("⚠️ Warning: More old reservations than new ones in database")
    
except Exception as e:
    print(f"❌ Reservation details insertion failed: {e}")
    traceback.print_exc()

Retrieved 4503 reservation IDs from database
✅ Inserted 9037 reservation details


## Verification

Let's verify the data was inserted correctly by counting the final row totals.

In [83]:
with engine.connect() as conn:
    tables = [
        'description', 'author', 'category', 'member', 'staff', 'book', 
        'book_copy', 'book_author', 'book_category', 'reservation', 'reservation_details'
    ]
    print("--- Final Row Counts ---")
    for table in tables:
        count = pd.read_sql(text(f"SELECT COUNT(*) FROM {table}"), conn).iloc[0, 0]
        print(f"Total rows in {table.ljust(20)}: {count}")

--- Final Row Counts ---
Total rows in description         : 2500
Total rows in author              : 4460
Total rows in category            : 802
Total rows in member              : 3000
Total rows in staff               : 100
Total rows in book                : 2500
Total rows in book_copy           : 5000
Total rows in book_author         : 6450
Total rows in book_category       : 5964
Total rows in reservation         : 4503
Total rows in reservation_details : 9037
